In [147]:
# %load Grid-testing.py
import tkinter as tk
from tkinter import ttk
import numpy as np

class Menubar(tk.Menu):
    def __init__(self,parent,*args,**kwargs):
        self.parent=parent
        tk.Menu.__init__(self,parent,*args,**kwargs)

        # initialise menu vars
        self.menu_file=tk.Menu(self)
        self.menu_simulation=tk.Menu(self)
        self.menu_about=tk.Menu(self)

        self.menu_file.add_command(label="Save", command=self.save_gridfile)
        self.menu_file.add_command(label="Open", command=self.open_gridfile)

        #collect them in a list
        menus = ((self.menu_file,"File"),
                 (self.menu_simulation,"Simulation"),
                 (self.menu_about,"About"))
        # quickly deploy the menus
        for i,(var,lab) in enumerate(menus):
            self.add_cascade(menu=var,label=lab)
    def save_gridfile(self):
        pass
    def open_gridfile(self):
        pass


class Cell(tk.Button):
    def __init__(self,parent,*args,**kwargs):
        tk.Button.__init__(self, parent, bg="white",activebackground="white",
                           relief="flat", overrelief="flat",
                           command=self.change_state,
                           *args, **kwargs)
    def change_state(self):
        if self.cget('bg') == 'white':
            self.configure(bg='black')
            self.configure(activebackground='black')
        else:
            self.configure(bg='white')
            self.configure(activebackground='white')


class App(tk.Canvas):
    def __init__(self, parent, n_rows, n_cols, *args, **kwargs):
        self.parent = parent
        tk.Canvas.__init__(self,parent, *args, **kwargs)

        self.menubar = Menubar(self)
        self.parent['menu'] = self.menubar

        self.n_rows = n_rows
        self.n_cols = n_cols

        self.cells = list()
        for i in range(0,self.n_rows*self.n_cols):
            self.cells.append(Cell(self,text=str(i)))
            row,col = divmod(i,self.n_cols)
            self.cells[i].grid(sticky='nsew', row=row, column=col)
        self.grid_matrix = np.ones((n_rows,n_cols))
        self.animate()
    def animate(self):
        r = np.random.randint(0,len(self.cells))
        self.cells[r].change_state()
        row,col = divmod(r,self.n_cols)
        # will have to loop through whole matrix
        # to include manually changed cell objects.
        if (self.cells[r].cget('bg') == 'black'):
            self.grid_matrix[row,col] = 0
        else:
            self.grid_matrix[row,col] = 1
        self.after(5000,self.animate)


if __name__ == "__main__":
    root = tk.Tk()
    root.title("Grid")
    root.resizable(tk.FALSE,tk.FALSE)
    root.option_add('*tearOff', tk.FALSE)
    main = App(root,n_rows=10,n_cols=10).grid(column=0,row=0,sticky='nesw')
    root.mainloop()

In [148]:
np.random.randint(1,100)

11

In [149]:
a = list()
a.append('1')
a.append('2')

In [150]:
a

['1', '2']

In [151]:
A = np.array([[1,0,0,1],[1,1,0,1],[1,0,0,0],[1,0,1,1]])
print(A)

[[1 0 0 1]
 [1 1 0 1]
 [1 0 0 0]
 [1 0 1 1]]


In [152]:
def incase(A):
    ra,ca = A.shape
    B = np.zeros((ra+2,ca+2))
    print(B)
    rb,cb = B.shape
    for i in range(1,rb-1):
        for j in range(1,cb-1):
            B[i,j] = A[i-1,j-1]
    B[1:rb-1,0] = A[:,ca-1]
    B[1:rb-1,cb-1] = A[:,0]
    B[0,1:cb-1] = A[ra-1,:]
    B[rb-1,1:cb-1] = A[0,:]
    
    B[0,0] = A[ra-1,ca-1]
    B[0,cb-1] = A[ra-1,0]
    B[rb-1,cb-1] = A[0,0]
    B[rb-1,0] = A[0,ra-1]
    print(B)

In [153]:
incase(A)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[1. 1. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1.]
 [1. 1. 1. 0. 1. 1.]
 [0. 1. 0. 0. 0. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1.]]


In [154]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


In [155]:
from dynamic_ext import *
import numpy as np

A = np.array([[0,1,1,1],[0,1,1,1],[0,1,1,1],[0,0,0,0]])

G = Grid(A)
print A
while(A.sum() > 0):
    A = G.get_next_gen(5)
    print A

[[0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 0 0]]
[[0 0 1 0]
 [1 1 1 1]
 [0 0 1 0]
 [0 0 1 0]]
[[0 1 0 1]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 0 0]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [3]:
import tkinter as tk
from tkinter import ttk
import numpy as np
from cellular_automaton import *

class App(tk.Canvas):
    def __init__(self, parent, n_rows, n_cols, initial_grid, threshold, *args, **kwargs):
        self.parent = parent
        tk.Canvas.__init__(self,parent, *args, **kwargs)
        
        self.threshold=threshold

        self.n_rows = n_rows
        self.n_cols = n_cols
        
        self.grid_matrix = initial_grid

        self.cells = list()
        for i in range(0,self.n_rows*self.n_cols):
            self.cells.append(Cell(self))
            row,col = divmod(i,self.n_cols)
            self.cells[i].grid(sticky='nsew', row=row, column=col)
            if (self.grid_matrix[row,col]==1):
                self.cells[i].configure(bg='white')
                self.cells[i].configure(activebackground='white')
            else:
                self.cells[i].configure(bg='black')
                self.cells[i].configure(activebackground='black')
        print(self.grid_matrix)
        self.Grid_obj = Grid(self.grid_matrix)
        
        self.menubar = Menubar(self)
        self.parent['menu'] = self.menubar
        
        #self.animate()
    def next_grid(self):
        self.grid_matrix = self.Grid_obj.get_next_gen(self.threshold)
        self.update_cells()
    def update_cells(self):
        print("u")
        for i in range(0,self.n_rows*self.n_cols):
            row,col = divmod(i,self.n_cols)
            if (self.grid_matrix[row,col]==1):
                if self.cells[i].cget('bg')=='black':
                    self.cells[i].change_state(to_state=1)
            else:
                if self.cells[i].cget('bg')=='white':
                    self.cells[i].change_state(to_state=0)
    def animate(self):
        self.update_cells()
        prev = (self.grid_matrix).copy()
        print(prev)
        self.next_grid()
        if (np.array_equal(prev,self.grid_matrix)):
            return
        self.after(750,self.animate)
    def update_grid(self):
        print("Ugrid")
        for i in range(0,self.n_rows*self.n_cols):
            row,col = divmod(i,self.n_cols)
            if self.cells[i].cget('bg') == 'white':
                self.grid_matrix[row,col] = 1
            elif self.cells[i].cget('bg') == 'black':
                self.grid_matrix[row,col] = 0
        self.Grid_obj = Grid(self.grid_matrix)
        print(self.grid_matrix)

class Menubar(tk.Menu, App):
    def __init__(self,parent,*args,**kwargs):
        self.parent=parent
        tk.Menu.__init__(self,parent,*args,**kwargs)
        # initialise menu vars
        self.menu_file=tk.Menu(self)
        self.menu_simulation=tk.Menu(self)
        self.menu_about=tk.Menu(self)

        self.menu_file.add_command(label="Save", command=self.save_gridfile)
        self.menu_file.add_command(label="Open", command=self.open_gridfile)
        
        self.menu_simulation.add_command(label="Run", command=lambda: parent.animate())
        self.menu_simulation.add_command(label="Update", command=lambda: parent.update_grid())
        #collect them in a list
        menus = ((self.menu_file,"File"),
                 (self.menu_simulation,"Simulation"),
                 (self.menu_about,"About"))
        # quickly deploy the menus
        for i,(var,lab) in enumerate(menus):
            self.add_cascade(menu=var,label=lab)
    def save_gridfile(self):
        pass
    def open_gridfile(self):
        pass

class Cell(tk.Button, App):
    def __init__(self,parent,*args,**kwargs):
        tk.Button.__init__(self, parent, bg="white",activebackground="white",
                           relief="flat", overrelief="flat",
                           command=self.change_state,
                           *args, **kwargs)
        self.parent = parent
    def change_state(self, to_state=None):
        if (self.cget('bg') == 'white' or to_state == 0) :
            self.configure(bg='black')
            self.configure(activebackground='black')
        elif (self.cget('bg') == 'black' or to_state == 1):
            self.configure(bg='white')
            self.configure(activebackground='white')
        if (to_state == None):
            self.parent.update_grid()
        #if (to_state==None):
        #    self.parent.update_cells()

n=20                   
A =  np.random.randint(0,2,(n,n))

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Grid")
    root.resizable(tk.FALSE,tk.FALSE)
    root.option_add('*tearOff', tk.FALSE)
    main = App(root,n_rows=n,n_cols=n, initial_grid=A,threshold=4).grid(column=0,row=0,sticky='nesw')
    root.mainloop()

[[0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 0 0 1]
 [0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 1 0]
 [1 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 0]
 [1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1]
 [1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1 1 0]
 [1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0]
 [0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1]
 [1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0]
 [0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1]
 [1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1]
 [1 0 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 1 1]
 [1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 0]
 [0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0]
 [1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 0 0]
 [1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1 0 1 0]
 [1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0 1]
 [1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1]
 [0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1]]


In [47]:
# %load Grid-testing.py
import tkinter as tk
from tkinter import ttk
import numpy as np
from cellular_automaton import *

n=20                   
A =  np.random.randint(0,2,(n,n))
Grid_obj = Grid(A)

Dynamic_windows = []

class App(tk.Frame):
    def __init__(self, parent, n_rows, n_cols, threshold, *args, **kwargs):
        self.parent = parent
        tk.Frame.__init__(self,parent, *args, **kwargs)
        self.controls = tk.Frame(self).grid(sticky='nw',row=0,column=0,columnspan=2,rowspan=2)
        
        self.threshold=threshold

        self.n_rows = n_rows
        self.n_cols = n_cols
        
        self.grid_matrix = A

        self.cells = list()
        for i in range(0,self.n_rows*self.n_cols):
            self.cells.append(Cell(self))
            row,col = divmod(i,self.n_cols)
            self.cells[i].grid(sticky='nsew', row=row, column=col)
            if (self.grid_matrix[row,col]==1):
                self.cells[i].configure(bg='white')
                self.cells[i].configure(activebackground='white')
            else:
                self.cells[i].configure(bg='black')
                self.cells[i].configure(activebackground='black')
        
        self.menubar = Menubar(self)
        self.parent['menu'] = self.menubar
        
        self.speed = 750
        
        self.speed_scale = tk.Scale(self.controls, orient=tk.VERTICAL, from_=0.1, to_=10,variable=self.speed, label="Seconds",resolution=0.1)
        self.speed_scale.grid(sticky='nw',row=0,column=n, rowspan=10)
        self.speed_scale.set(self.speed)
        #self.animate()
    def next_grid(self):
        self.grid_matrix = Grid_obj.get_next_gen(self.threshold)
        self.update_cells()
    def update_cells(self):
        print("u")
        for i in range(0,self.n_rows*self.n_cols):
            row,col = divmod(i,self.n_cols)
            if (self.grid_matrix[row,col]==1):
                if self.cells[i].cget('bg')=='black':
                    self.cells[i].change_state(to_state=1)
            else:
                if self.cells[i].cget('bg')=='white':
                    self.cells[i].change_state(to_state=0)
    def animate(self):
        self.update_cells()
        prev = (self.grid_matrix).copy()
        print(prev)
        self.next_grid()
        if (np.array_equal(prev,self.grid_matrix)):
            return
        self.after(int(1000*self.speed_scale.get()),self.animate)
    def update_grid(self):
        print("Ugrid")
        for i in range(0,self.n_rows*self.n_cols):
            row,col = divmod(i,self.n_cols)
            if self.cells[i].cget('bg') == 'white':
                self.grid_matrix[row,col] = 1
            elif self.cells[i].cget('bg') == 'black':
                self.grid_matrix[row,col] = 0
        print(type(Grid_obj))
        Grid_obj.set_grid(self.grid_matrix)
        print(self.grid_matrix)

class Menubar(tk.Menu, App):
    def __init__(self,parent,*args,**kwargs):
        self.parent=parent
        tk.Menu.__init__(self,parent,*args,**kwargs)
        
        self.about_window=None
        
        # initialise menu vars
        self.menu_file=tk.Menu(self)
        self.menu_simulation=tk.Menu(self)
        self.add_command(label="About",command=lambda: self.about())

        self.menu_file.add_command(label="Save", command=lambda: self.save_gridfile)
        self.menu_file.add_command(label="Open", command=lambda: self.open_gridfile)
        
        self.menu_simulation.add_command(label="Run", command=lambda: parent.animate())
        self.menu_simulation.add_command(label="Update", command=lambda: parent.update_grid())
        
        #collect them in a list
        menus = ((self.menu_file,"File"),
                 (self.menu_simulation,"Simulation"))
        # quickly deploy the menus
        for i,(var,lab) in enumerate(menus):
            self.add_cascade(menu=var,label=lab)
    def save_gridfile(self):
        pass
    def open_gridfile(self):
        pass
    def about(self):
        if self.about_window not in Dynamic_windows:
            self.about_window = tk.Toplevel()
            self.about_window.title("About")
            s = """This is a cellular automaton simulator.
            
https://en.wikipedia.org/wiki/Cellular_automaton
            
-GUI written in python 2.7 Tkinter.
-Simulator engine written in C++ with boost.
-Harvey Devereux.
-harveydevereux@googlemail.com.
-version 0.2
"""
            info = tk.Label(self.about_window, text=s)
            info.grid(column=0,row=0)
            Dynamic_windows.append(self.about_window)
            
            

class Cell(tk.Button, App):
    def __init__(self,parent,*args,**kwargs):
        tk.Button.__init__(self, parent, bg="white",activebackground="white",
                           relief="flat", overrelief="flat",
                           command=self.change_state,
                           *args, **kwargs)
        self.parent = parent
    def change_state(self, to_state=None):
        if (self.cget('bg') == 'white' or to_state == 0) :
            self.configure(bg='black')
            self.configure(activebackground='black')
        elif (self.cget('bg') == 'black' or to_state == 1):
            self.configure(bg='white')
            self.configure(activebackground='white')
        if (to_state == None):
            self.parent.update_grid()
        #if (to_state==None):
        #    self.parent.update_cells()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Grid")
    root.resizable(tk.FALSE,tk.FALSE)
    root.option_add('*tearOff', tk.FALSE)
    main = App(root,n_rows=n,n_cols=n,threshold=4).grid(column=0,row=0,sticky='nesw')
    root.mainloop()

u
[[1 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0]
 [0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1]
 [0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 1 1]
 [1 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1]
 [0 0 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1]
 [0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0]
 [1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 0]
 [0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0]
 [0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 0]
 [0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0]
 [0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0]
 [0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0]
 [1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 0]
 [1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1]
 [1 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0]
 [1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 0]
 [1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
 [1 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 0]]
u
u
[[1 1 1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1]
 [1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 1]
 [1 1 0 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1]
 [1 

In [145]:
A = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

for i in range(0,A.shape[0]*A.shape[1]):
    row,col = divmod(i,A.shape[1])
    print(row,col)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(3, 0)
(3, 1)
(3, 2)
(3, 3)


In [135]:
B = np.array([[0,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
np.array_equal(A,B)

False

In [46]:
b = np.random.randint(0,2,(10,10))
b

array([[0, 0, 1, 0, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 1, 1, 0, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 1, 0, 1]])

In [8]:
f = tk.Frame()
t = tk.Frame()
windows = [f]
f not in windows
windows.append(t)

windows

[<Tkinter.Frame instance at 0x7fdec866bf38>,
 <Tkinter.Frame instance at 0x7fdec866bc68>]

'This is a cellular automaton simulator.\n\n    -GUI written in python 2.7 Tkinter.\n'

In [37]:
s= """glflsd
    skkcn
dlasmads
odsam"""

In [38]:
print(s)

glflsd
    skkcn
dlasmads
odsam
